In [17]:
import pandas as pd
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os
load_dotenv()
from langchain.chains import LLMChain
import json
from langchain.prompts import (
    ChatPromptTemplate,
)
from langchain.llms import OpenAI




In [18]:
# HIER BITTE MIT BEIDEN PATHS EINMAL DURCHLAUFEN LASSEN

GENERATIONS_PATH = "../../datasets/error_analysis/auto_gpt3_gpt3_50_50_generations_merged.csv"
# GENERATIONS_PATH = "../../datasets/error_analysis/zero_shot_cot_50_50_merged.csv"
MODEL = "gpt-4-0613"


In [19]:
data = pd.read_csv(GENERATIONS_PATH)

data

,Unnamed: 0,traceId,model,startTime,endTime,prompt,completion,eval,predicted_label,label,error,similarity,confidence,structured_explanation_v2
0,0,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T12:39:55.032Z,2023-10-20T12:40:03.460Z,Q: Are the following two products the same?\nP...,"First, we can compare the titles and descripti...",no match,False,True,FN,20%,90%,"Based on the information provided, the attribu..."
1,1,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T12:39:42.734Z,2023-10-20T12:39:53.607Z,Q: Are the following two products the same?\nP...,"First, we can compare the titles and descripti...",The two products may match.,True,True,NONE,90%,80%,"[{""attribute"":""brand"",""importance"":""0.10"",""val..."
2,2,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T12:39:28.325Z,2023-10-20T12:39:40.801Z,Q: Are the following two products the same?\nP...,"First, we can compare the titles and descripti...",Match.,True,True,NONE,90%,80%,NaN
3,3,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T12:39:10.835Z,2023-10-20T12:39:26.196Z,Q: Are the following two products the same?\nP...,"First, we can compare the titles and descripti...",Match.,True,True,NONE,60%,80%,NaN
4,4,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T12:38:55.776Z,2023-10-20T12:39:08.718Z,Q: Are the following two products the same?\nP...,"First, we can compare the titles of the two pr...",The two products may match.,True,True,NONE,90%,80%,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,216,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T11:59:42.555Z,2023-10-20T11:59:51.404Z,Q: Are the following two products the same?\nP...,"First, we can compare the titles and descripti...",no match,False,False,NONE,0%,100%,NaN
217,217,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T11:59:28.660Z,2023-10-20T11:59:41.048Z,Q: Are the following two products the same?\nP...,"First, we can compare the titles and descripti...",no match,False,False,NONE,25%,90%,NaN
218,218,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T11:59:17.151Z,2023-10-20T11:59:26.845Z,Q: Are the following two products the same?\nP...,"First, we can compare the titles and descripti...",no match,False,False,NONE,0%,100%,NaN
219,219,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T11:59:04.839Z,2023-10-20T11:59:15.798Z,Q: Are the following two products the same?\nP...,"First, we can compare the titles and descripti...",no match,False,False,NONE,25%,80%,NaN


In [20]:
task_structured_explanation = """Explain your decision in a structured format, listing the attributes that you compared for reaching your decision. Each attribute should be accompanied by the attribute values and a score between 0 and 1 that shows the importance of the attribute for the decision. If an attribute only occurs in one item, specify the value of that attribute for the other item as "missing". The attributes should not be just the Title or Description but more specific attributes like Brand, Model, Color, etc. 
The format should exactly match the following example output: 
[{"attribute":"brand","importance":"0.05","values":["Logitech","Logitech"]},{"attribute":"model","importance":"0.95" "values":["G500","MX Master 3S"]},{"attribute":"color","importance":"0.00" "values":["missing","Graphite"]}]"""

task_structured_explanation_v2 = """Explain your decision in a structured format, listing the attributes that you compared for reaching your decision. Each attribute should be accompanied by the attribute values and a score between 0 and 1 that shows the importance of the attribute for the decision. If an attribute only occurs in one item, specify the value of that attribute for the other item as "missing". The attributes should not be just the Title or Description but more specific attributes like Brand, Model, Color, etc. 
The format should exactly match the following example output: 
[{"attribute":"brand","importance":"0.05","values":["Logitech","Logitech"]},{"attribute":"model","importance":"0.95", "values":["G500","MX Master 3S"]},{"attribute":"color","importance":"0.00", "values":["missing","Graphite"]}]"""



STEP_EXPLANATION_STRUCTURED_PROMPT = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful AI."),
            ("human", "{user_prompt}"),
            ("ai", "{ai_answer}"),
            ("human", "{task_structured_explanation}"),
        ])




In [21]:
# if "structured_explanation" not in data.columns:
    # data["structured_explanation"] = ""
# if "structured_explanation_v2" not in data.columns:
    # data["structured_explanation_v2"] = ""
if "structured_explanation_v2_gpt4" not in data.columns:
    data["structured_explanation_v2_gpt4"] = ""

In [22]:
def get_structured_explanation(start, end):
    if (MODEL == "gpt-3.5-turbo-instruct"):
        llm = OpenAI(model_name=MODEL, temperature=0)
    else:
        llm = ChatOpenAI(model_name=MODEL, temperature=0)
    chain = LLMChain(llm=llm, prompt=STEP_EXPLANATION_STRUCTURED_PROMPT)
    for i in range(start, end):
        print(i)
        whole_prompt = data["prompt"][i]
        user_prompt = "Q:" + whole_prompt.split("Q:")[-1]
        ai_answer = data["completion"][i]
        if data["predicted_label"][i]:
            prediction = "Match"
        else:
            prediction = "No Match"
        ai_answer = ai_answer + "\n" + prediction
        try:
            output = chain.run(user_prompt=user_prompt, ai_answer=ai_answer, task_structured_explanation=task_structured_explanation_v2)
        except:
            output = "error"
            i = i - 1
            continue
        print(STEP_EXPLANATION_STRUCTURED_PROMPT.format(user_prompt=user_prompt, ai_answer=ai_answer, task_structured_explanation=task_structured_explanation_v2))
        print("----")
        print(data["label"][i])
        print(output)
        print("-----------")
        data["structured_explanation_v2_gpt4"][i] = output



In [23]:
get_structured_explanation(0, len(data))

0
System: You are a helpful AI.
Human: Q: Could you please tell me if the following two products are the same one?
Product 1:
Title: Traveler's Lined Notebook Refill - Regular Size
Description: Think about the days you went to school. Based on people’s past experience as a student, we receive comments saying that this is the most easy type of notebook to use. The lines are printed with an interval of 6.5mm, and the paper is our originally m
Product 2:
Title: Traveler's Company Traveler's Company Notebook Accessories
Description: none
A: Let's think step by step!
AI: First, we can compare the titles and descriptions of the two products. The first product is a refill for a lined notebook in regular size, with a specific interval between the lines and made by a specific company. The second product is a set of notebook accessories made by the same company, but with no specific details mentioned. \nBased on this information, we can conclude that the two products are not the same. They are b

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_629/2085426600.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["structured_explanation_v2_gpt4"][i] = output


In [24]:
data

,Unnamed: 0,traceId,model,startTime,endTime,prompt,completion,eval,predicted_label,label,error,similarity,confidence,structured_explanation_v2,structured_explanation_v2_gpt4
0,0,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T12:39:55.032Z,2023-10-20T12:40:03.460Z,Q: Are the following two products the same?\nP...,"First, we can compare the titles and descripti...",no match,False,True,FN,20%,90%,"Based on the information provided, the attribu...","Based on the information provided, the attribu..."
1,1,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T12:39:42.734Z,2023-10-20T12:39:53.607Z,Q: Are the following two products the same?\nP...,"First, we can compare the titles and descripti...",The two products may match.,True,True,NONE,90%,80%,"[{""attribute"":""brand"",""importance"":""0.10"",""val...",
2,2,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T12:39:28.325Z,2023-10-20T12:39:40.801Z,Q: Are the following two products the same?\nP...,"First, we can compare the titles and descripti...",Match.,True,True,NONE,90%,80%,NaN,
3,3,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T12:39:10.835Z,2023-10-20T12:39:26.196Z,Q: Are the following two products the same?\nP...,"First, we can compare the titles and descripti...",Match.,True,True,NONE,60%,80%,NaN,
4,4,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T12:38:55.776Z,2023-10-20T12:39:08.718Z,Q: Are the following two products the same?\nP...,"First, we can compare the titles of the two pr...",The two products may match.,True,True,NONE,90%,80%,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,216,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T11:59:42.555Z,2023-10-20T11:59:51.404Z,Q: Are the following two products the same?\nP...,"First, we can compare the titles and descripti...",no match,False,False,NONE,0%,100%,NaN,
217,217,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T11:59:28.660Z,2023-10-20T11:59:41.048Z,Q: Are the following two products the same?\nP...,"First, we can compare the titles and descripti...",no match,False,False,NONE,25%,90%,NaN,
218,218,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T11:59:17.151Z,2023-10-20T11:59:26.845Z,Q: Are the following two products the same?\nP...,"First, we can compare the titles and descripti...",no match,False,False,NONE,0%,100%,NaN,
219,219,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T11:59:04.839Z,2023-10-20T11:59:15.798Z,Q: Are the following two products the same?\nP...,"First, we can compare the titles and descripti...",no match,False,False,NONE,25%,80%,NaN,


In [25]:
data.to_csv(GENERATIONS_PATH, index=False)
